In [25]:
import numpy as np

from BinaryReader import BinaryReader
import Preprocessor
from models import classiRaw3D
import importlib

In [29]:
importlib.reload(Preprocessor)

<module 'Preprocessor' from '/home/julius/dataspellprojects/oct-classifier/Preprocessor.py'>

In [35]:
reader = BinaryReader()  # TODO: Normalizer
dataset = reader.create_training_datasets()
preprocesser = Preprocessor.Preprocessor(dataset)

In [50]:
preprocesser.standardize()
print(preprocesser.mean, preprocesser.std)
preprocesser.calc_moments()
print(preprocesser.mean, preprocesser.std)

Calculating Mean and Standard Deviation of whole Dataset. This could take a while
15.726354266820328 15.157980211665121
Calculating Mean and Standard Deviation of whole Dataset. This could take a while
-4.057625793914698e-08 1.0000000221005987


In [ ]:
model = classiRaw3D(dataset.element_spec[0].shape)
model.save('savedModels/first')